# Example: comparing Broyden to Automatic Differentiation
By T. Fitzgerald

We'll be solving the same system of equations that we worked on during the previous class
$$ \frac{x^2}{4} + y^2 = 1 $$
$$ y = 0.7x^4 + 0.1x^3 - 2.5x^2 + 1 $$

I'll use several packages to help with this: [ForwardDiff.jl](http://www.juliadiff.org/ForwardDiff.jl/stable/) and [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl).  ForwardDiff is an automatic differentiation library that very efficiently (and robustly) computes the derivatives of a function.  I'm using BenchmarkTools to help gauge how expensive the relative methods are.

In [1]:
using Pkg
Pkg.activate(".")
using LinearAlgebra
using ForwardDiff
using BenchmarkTools

Let's define the same functions as before

In [2]:
p(x) = 0.7x^4 + 0.1x^3 - 2.5x^2 + 1
f1(z) = z[1]^2/4 + z[2]^2 - 1
f2(z) = p(z[1]) - z[2]
f(z) = [f1(z), f2(z)]
;

and the full Jacobian, for use with Newton's method

In [3]:
function Jt(z)
    x = z[1]
    y = z[2]
    Jt = zeros(2,2)
    Jt[1,1] = x/2
    Jt[1,2] = 2y
    Jt[2,1] = 0.7*4*x^3 + 0.1*3*x^2 - 2.5*2*x
    Jt[2,2] = -1
    return Jt
end
Jt([1,1])

2×2 Array{Float64,2}:
  0.5   2.0
 -1.9  -1.0

I can use the `jacobian` function of ForwardDiff to compute the multidimensional derivative of the residuals `f`, evaluated at a point `x` magically.  I'll wrap that call into a function, so I can just call that later.

In [4]:
∇f(x) = ForwardDiff.jacobian( f, x )

∇f (generic function with 1 method)

Calling the function gives the *same* result as before

In [5]:
∇f([1,1])

2×2 Array{Float64,2}:
  0.5   2.0
 -1.9  -1.0

In [6]:
norm( ∇f([-1,1]) - Jt([-1.,1]) )

0.0

## Broyden Updates

In [7]:
function myBroyden(x0, J0; tol = 1e-8, max_iter = 50)
    iter = 0
    flag = 0

    # initial step
    D = inv(J0)
    d = -D*f(x0)
    x1 = x0 + d
    F = f(x1)
    
    while flag == 0

        u = D*F
        c = d'*( d + u )
        D -= 1/c*(u*d')*D   

        iter += 1

        d = -D*F
        x1 = x1 + d
        F = f(x1)
        
        err = norm(F)

        if err <= tol
            flag = 1

        elseif iter >= max_iter
            flag = -1
            
        end  

    end
    
    return x1, flag, iter
    
end
;

## Newton's Method

In [8]:
function myNewton(x0; tol = 1e-8, max_iter = 50)
    iter = 0
    flag = 0

    x1 = copy(x0)
    F = f(x1)
    while flag == 0

        iter += 1
        J = Jt(x1) # using the full jacobian
        x1 -= J\F
        F = f(x1)
        
        err = norm(F)

        if err <= tol
            flag = 1

        elseif iter >= max_iter
            flag = -1
            
        end  

    end
    
    return x1, flag, iter
    
end

myNewton (generic function with 1 method)

## Newton's method with approximate Jacobian

In [9]:
function myADNewton(x0; tol = 1e-8, max_iter = 50)
    iter = 0
    flag = 0

    x1 = copy(x0)
    F = f(x1)
    while flag == 0

        iter += 1
        J = ∇f(x1) # this is the AD step
        x1 -= J\F
        F = f(x1)
        
        err = norm(F)

        if err <= tol
            flag = 1

        elseif iter >= max_iter
            flag = -1
            
        end  

    end
    
    return x1, flag, iter
    
end
;

## Testing time
First, let's assume the same initial guess for all three methods

In [10]:
x0 = [2,-2];

In [11]:
myNewton(x0)

([1.53561, -0.640681], 1, 5)

In [12]:
myADNewton(x0)

([1.53561, -0.640681], 1, 5)

In [13]:
# Compute Jacobian by finite differences
ε = 1e-2
J0 = zeros(2,2)
J0[:,1] = ( f(x0 + ε*[1,0]) - f(x0) )/ε
J0[:,2] = ( f(x0 + ε*[0,1]) - f(x0) )/ε
myBroyden(x0, J0)

([1.53561, -0.640681], 1, 10)

We can see that all 3 methods converged.  Also, the number of steps for full Newton and AD Newton are the same. Broyden was only slightly slower with several additional steps.  But we know that, in general, the cost per step is lower so the overall costs are all nearly the same.  

To help look in to the timings of each method, I'm going to use the BenchmarkTools.jl package.

In [14]:
@benchmark myNewton(x0)

BenchmarkTools.Trial: 
  memory estimate:  3.34 KiB
  allocs estimate:  38
  --------------
  minimum time:     2.944 μs (0.00% GC)
  median time:      4.500 μs (0.00% GC)
  mean time:        8.091 μs (19.45% GC)
  maximum time:     8.337 ms (99.93% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [15]:
@benchmark myADNewton(x0)

BenchmarkTools.Trial: 
  memory estimate:  4.98 KiB
  allocs estimate:  63
  --------------
  minimum time:     6.360 μs (0.00% GC)
  median time:      7.940 μs (0.00% GC)
  mean time:        12.618 μs (19.75% GC)
  maximum time:     17.408 ms (99.82% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [16]:
@benchmark myBroyden(x0, J0)

BenchmarkTools.Trial: 
  memory estimate:  12.22 KiB
  allocs estimate:  121
  --------------
  minimum time:     6.460 μs (0.00% GC)
  median time:      7.860 μs (0.00% GC)
  mean time:        13.139 μs (26.69% GC)
  maximum time:     16.834 ms (99.88% GC)
  --------------
  samples:          10000
  evals/sample:     5

The results show that Broyden is slower.  It needed more steps, so this makes perfect sense.  However, what this test problem does not show is how as the problem size increases that Broyden becomes very competitive.  Since the approximate update is to the *inverse* of the Jacobian, no linear-system needs to be inverted and that can become a dominating cost as the problem size increases.